#Import Libraries

In [ ]:
!pip install pynauty

     |████████████████████████████████| 1.9 MB 32.1 MB/s 
  Created wheel for pynauty: filename=pynauty-1.0.2-cp37-cp37m-linux_x86_64.whl size=319208 sha256=57ebdd823939213dceff4ff778a63bb414d4f19e2a9d32d95906a7caa9102c16
  Stored in directory: /root/.cache/pip/wheels/62/5f/2b/1f298d1b74899fc4bba74440e892a178ce76d0226856e5896d
Successfully built pynauty


In [ ]:
import networkx as nx
import copy
import random
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pynauty as pyn

#Reading File

In [ ]:
def saveGmlFromTxt(name,path):
    file = open(path)
    G = nx.DiGraph()
    for line in file.readlines():
        x,y = map(int,list(line.split()))
        if(x == y):
            continue
        G.add_edge(x,y)
    
    nx.write_gml(G,name)

saveGmlFromTxt('littleRock.gml','/content/out.maayan-foodweb')

In [ ]:
def loadFile(path, Directed):
    if(Directed == False):
        G = nx.read_gml(path)
    else:
        G = nx.read_gml(path)
    return G

#Function to Generate Motifs with Given Size

In [ ]:
def getMotifs(k, Directed):
  if k == 1 :
    if Directed :
      G = nx.DiGraph()
      G.add_node(0)
      return [G]
    else :
      G = nx.Graph()
      G.add_node(0)
      return [G]
  else : 
    smaller_motifs = getMotifs(k - 1,Directed) 
   
    edges = k - 1 
    motifs = []
    motifs_degree = []

    if Directed : 
      edges *= 2

    for g in smaller_motifs :
      for mask in range(2**edges) :
        if mask == 0 :
          continue

        new_g = g.copy()
        
        for node in range(edges) : 
          if (mask & (1 << node)) > 0 :  
            if Directed : 
              if node >= k - 1 :
                new_g.add_edge(k - 1,node - (k - 1))
              else :
                new_g.add_edge(node,k - 1)
            else : 
              new_g.add_edge(node,k - 1)

        #check if new_g is isomorphis to all existing motifs
        motif_found = False
        cur_degree = sorted([d for n, d in new_g.degree()])

        for i,motif in enumerate(motifs) : 
          if cur_degree != motifs_degree[i] :
            continue 

          if nx.is_isomorphic(motif , new_g) : 
            motif_found = True
            break

        if motif_found == False : 
          motifs.append(new_g)
          motifs_degree.append(cur_degree)

    return motifs

#ESU & Rand-ESU Functions

In [ ]:
def EnumerateSubgraphs(G,k,q = 1,motifFind = 0):
    vertexList = list(G.nodes)
    for v in vertexList:
        Vext = set(filter(lambda x: x>v, list(G[v])))
        Vsub = set({v})
        neighborHood = set(filter(lambda x: x > v, set(G[v])))
        p = q**(1/k)
        extendSubgraph(G,Vsub,Vext,neighborHood,v,k,p,motifFind)

def extendSubgraph(G,Vsub, Vext, neighborHood, v, k, p , motifFind = 0):
    if(Vsub is None):
        return 
    if(len(Vsub) == k):
        cnt[0] += 1
        if(motifFind == 1):
            subG.append(Vsub)
        return
    while(len(Vext) != 0):
        w = Vext.pop()
        if(random.uniform(0,1) <= p):
            wNeighbor = set(filter(lambda x: x > v and x != w, set(G[w])))
            Nexcl = wNeighbor.difference(neighborHood)

            if(Nexcl is not None):
                Vext_dash = Vext.union(Nexcl)
            else:
                Vext_dash = copy.copy(Vext)

            if(wNeighbor is not None):
                neighborHood_dash = neighborHood.union(wNeighbor)
            else:
                neighborHood_dash = copy.copy(neighborHood)

            Vsub_dash = Vsub.union({w})
            extendSubgraph(G,Vsub_dash,Vext_dash,neighborHood_dash,v,k, p,motifFind)

    return 

#Functions for Counting Motifs

In [ ]:
def canonical(G):
    num_nodes = G.number_of_nodes()
    nodes = sorted(list(G.nodes))
    nk = {}
    for i in range(len(nodes)):
        nk[nodes[i]] = i

    g = pyn.Graph(num_nodes,directed = Directed)
    for x in list(G.nodes):
        tmp = [nk[y] for y in G[x]]
        g.connect_vertex(nk[x],tmp)
    return pyn.certificate(g)


from joblib import Parallel, delayed
def countMotifs(G,subG,motifs):
    binaries = []
    indx = {}

    i = 0
    for mo in motifs:
        px = canonical(mo)
        binaries.append(px)
        indx[px] = i
        i += 1

    # for x in subG:
    def process(low,high):
        cnts = [0 for x in range(len(motifs))]
        for i in range(low,high + 1):
            G_dash = G.subgraph(subG[i])
            px = canonical(G_dash)
            val = indx.get(px)
            if(val is not None):
                cnts[val] += 1
        return cnts

    ranges = []
    ln = len(subG)//4
    rem = len(subG)%4
    last = 0
    for job in range(4):
        if(job == 3):
            ranges.append((last,last + ln - 1 + rem))
        else:
            ranges.append((last, last + ln - 1))
        last = last + ln

    results = Parallel(n_jobs=4)(delayed(process)(rr[0],rr[1]) for rr in ranges)
    toRet = np.sum(results,axis = 0)
    return toRet

#Function for Getting Number of Motifs 

In [ ]:
# all motifs must be of same size, function returns expected number of motifs
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

def findExpectedMotifOccurences(G,motifSize,motifs,prob, exp):
    global cnt
    cnt = [0]
    if(Directed):
        Gu = G.to_undirected()
    else:
        Gu = G
    
    EnumerateSubgraphs(Gu,motifSize,motifFind=0)
    uniCnts = []
    uniLen = cnt[0]

    for exp in range(exp):
        subG.clear()
        cnt = [0]
        EnumerateSubgraphs(Gu,motifSize,prob,motifFind=1)

        lenSub = len(subG)
        cnts = countMotifs(G,subG,motifs)
        cnts = [x/lenSub * uniLen for x in cnts]
        uniCnts.append(cnts)

    uniCnts = np.array(uniCnts)

    meanOccurences = np.mean(uniCnts, axis=0)
    return meanOccurences


# Main

In [ ]:
# G can be Directed or undirecetrd
Directed = True
G = loadFile('/content/littleRock.gml', Directed)

In [ ]:
#Define Motif
motifSize = 4
motif = nx.DiGraph()
motif.add_edge(1,4)
motif.add_edge(1,3)
motif.add_edge(3,2)
motif.add_edge(4,2)

# motif1 = nx.DiGraph()
# motif1.add_edge(1,3)
# motif1.add_edge(1,4)
# motif1.add_edge(3,2)
# motif1.add_edge(4,2)

motifs = [motif]

#Get Real count
subG = []
sampling = 1
exp = 1

ans = findExpectedMotifOccurences(G, motifSize, motifs, sampling,exp)

#get randomised graph count
arr = []
for numGraphs in range(10):
    if(Directed):
        kin = list(d for n, d in G.in_degree())
        kout = list(d for n, d in G.out_degree())
        Gr = nx.directed_configuration_model(kin,kout,create_using=nx.DiGraph)

        subG = []
        tmp = findExpectedMotifOccurences(Gr, motifSize, motifs, sampling,exp)
        arr.append(tmp)
    else:
        k = list(d for n, d in G.degree())
        Gr = nx.configuration_model(k,create_using=nx.Graph)

        subG = []
        tmp = findExpectedMotifOccurences(Gr, motifSize, motifs, sampling,exp)
        arr.append(tmp)

arr = np.array(arr)

#get mean and sd
mn = np.mean(arr,axis=0)
sd = np.std(arr,axis = 0)

z_score = (ans - mn)/sd

In [ ]:
print(mn)
print(sd)
print(ans)
print(z_score)


[960.9]
[91.18163192]
[65123.]
[11.08885615]


2434136

# Rough

In [ ]:

# real_ans = [1026725,816765,131156,15004,11772,2576]
# # real_ans = np.array(real_ans)/sum(real_ans)
# ans1 = sorted(ans, reverse = True)

# for i in range(6):
#     print((ans1[i] - real_ans[i])/real_ans[i] * 100)

In [ ]:

# real_ans = [23279327,12013183,7294043,1672420,1543841,1290930,733663,422212,267279,97449,50722,39493,35808,32895,27981,22581,21043,11154,7834,4906,1711]
# real_ans = np.array(real_ans)/sum(real_ans)
# ans1 = sorted(ans, reverse = True)
# for i in range(21):
#     print((ans1[i] - real_ans[i])/real_ans[i] * 100)